In [17]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity, tensorboard_trace_handler

In [18]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

In [19]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True,on_trace_ready=tensorboard_trace_handler("profiles/")) as prof:
   with record_function("model_inference"):
      model(inputs)

In [14]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         2.41%       8.572ms        99.98%     355.983ms     355.983ms          -4 b    -106.30 Mb             1  
                     aten::conv2d         0.05%     178.000us        55.72%     198.395ms       9.920ms      47.37 Mb           0 b            20  
                aten::convolution         0.16%     554.000us        55.67%     198.217ms       9.911ms      47.37 Mb           0 b            20  
               aten::_convolution         0.11%     383.000us        55.51%     197.663ms       9.883ms      47.

In [8]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=30))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         2.41%       8.572ms        99.98%     355.983ms     355.983ms          -4 b    -106.30 Mb             1                                                                                []  
                     aten::conv2d         0.01%      38.000us        12.55%     

In [16]:
prof.export_chrome_trace("trace.json")